# FIT5196 Assessment 1

Student Name: Jaimon Thyparambil Thomas

Student ID: 29566428

Date:14-04-2019

Environment: Python 3.6.5 and Jupyter notebook Libraries used:

re (for regular expression)

json (for converting dictionary to json)

# Importing the Necessary Libraries

In [1]:
import re
from json import dumps

# Task 1: Parsing Raw Text Files

## Introduction

This assessment touches the very first step of analyzing textual data, i.e., extracting data
from semi-structured text files. Here the data-set that is to be processed contains
information about several units in the Monash University. Each data-set contains information about the unit, e.g., unit
code, unit title, synopsis, requirements, output, chief examiner, etc. (see test.txt). Here we will be
extracting the data and transform the data into the XML and JSON format.

Defining the Basic Class and Functions used for processing each unit blocks

In [2]:
#function used to check wheather the given value is NA or nor not
def isNA(value):
    if type(value)!= list:
        return value == "NA" or value == "TBA"
    return False

#Basic class block for every units
class UnitBlock:

    block_count = 0 #variable used to maintain the no of unit blocks created 
    #Order in which entries has to be processed while creating the XML File
    xml_seq_order = ["@id","title","synopsis","pre_requistics","prohibisions","requirements","outcomes","chief_examiners"]
    emptyCharSeq = ("NA","TBA") #Tuple used to maintain the list of variables used to denote empty values

    #initializer Function used to setting default values and initializing objects
    def __init__(self):
        UnitBlock.block_count += 1
        self.unit_id = "" 
        self.title = "NA" 
        self.synopsis = "NA" 
        self.pre_requistics = {} 
        self.prohibisions = {} 
        self.requirements = {}  
        self.outcomes = {} 
        self.chief_examiners = {}  
        self.block_dict = {} # value contains all the details of the unit as a dictionary
        return

    def getBlockDictionary(self):
        return self.block_dict

    def set_unit_id(self,value):
        self.unit_id = value
        return

    def set_title(self,value):
        self.title = value
        return
    
    def set_synopsis(self,value):
        self.synopsis = value
        return
    
    def set_pre_requistics(self,value):
        #Handling the case if the value is empty only NA is added else entry is made in a dictionary
        if not isNA(value):
            self.pre_requistics["pre_requistic"] = value
        else:
            self.pre_requistics = value 
        return

    def set_prohibisions(self,value):
        #Handling the case if the value is empty only NA is added else entry is made in a dictionary
        if not isNA(value):
            self.prohibisions["prohibision"] = value
        else:
            self.prohibisions = value
        return

    def set_requirements(self,value):
        #Handling the case if the value is empty only NA is added else entry is made in a dictionary
        if not isNA(value):
            self.requirements["requirement"] = value
        else:
            self.requirements = value
        return

    def set_outcomes(self,value):
        #Handling the case if the value is empty only NA is added else entry is made in a dictionary
        if not isNA(value):
            self.outcomes["outcome"] = value
        else:
            self.outcomes = value
        return

    def set_chief_examiners(self,value):
        #Handling the case if the value is empty only NA is added else entry is made in a dictionary
        if not isNA(value):
            self.chief_examiners["chief_examiner"] = value
        else:
            self.chief_examiners = value
        return

    #Function used to check if the unit block is empty or not
    def isEmpty(self):
        return len(self.unit_id) == 0

    #function used to set all the values into a dictionary
    def configure_dict(self):
        self.block_dict["@id"] = self.unit_id
        self.block_dict["title"] = self.title
        self.block_dict["synopsis"] = self.synopsis
        self.block_dict["pre_requistics"] = self.pre_requistics
        self.block_dict["prohibisions"] = self.prohibisions
        self.block_dict["requirements"] = self.requirements
        self.block_dict["outcomes"] = self.outcomes
        self.block_dict["chief_examiners"] = self.chief_examiners
        return

    #Function Used to Write all the values in the unitBlocks in to a xml File
    def writeToXML(unit_blocks = [],writeFileName=""):
        outFile = open(writeFileName,"w+")
        outFile.write("<?xml version=\"1.0\" encoding=\"UTF-8\" ?>")
        outFile.write("\n<units>")
        for each in unit_blocks:
            block = each.getBlockDictionary()
            tempStr = "\n<unit id='"+block["@id"]+"'>"
            for order in UnitBlock.xml_seq_order:
                if order == "@id":
                    #Since unit id follows a specfic syntax like <unit id='unitid'>
                    #It is been handled seperately
                    continue
                tempStr += ("\n<" + order + ">")
                value = block[order]
                if (type(value) == dict):
                    for key,subDictValues in value.items():
                        if(type(subDictValues) == str ):
                            #strip is used to handle cases such as NA and " NA "
                            #If it is NA or TBA value is directly added to the xml else value is added to an array
                            if(subDictValues.strip() in UnitBlock.emptyCharSeq):
                                tempStr += subDictValues
                                continue
                            else:
                                subDictValues = [subDictValues]
                        tempStr += "\n"
                        for val in subDictValues:
                            tempStr +=("<" + key + ">")
                            tempStr += val
                            tempStr +=("</" + key + ">")
                else:
                    tempStr += value
                tempStr += ("</" + order + ">")
            tempStr += "\n</unit>"
            outFile.write(tempStr)
        outFile.write("\n</units>")
        outFile.close()
        return

    #function used to convert the entire unit blocks to dictionaries value wits key units
    def to_dict(unit_blocks = []):
        units = []
        for each in unit_blocks:
            units.append(each.getBlockDictionary())
        res ={}
        res["unit"] = units
        final_res = {}
        final_res["units"] = res
        return final_res

In [3]:
def isEmpty(str):
    return len(str) == 0

#function Used to remove links in the value
def handleLinks(res_array):
    new_res = []
    for each in res_array:
        new_res.append(re.sub(r'<[\w\s=\>"/.!:-]+>', "", each))
    return new_res

#function used to handle single value cases
def handleSingleValueCase(value,is_cheif_examiner = False, isXML = False):
    if not isEmpty(value):
        if len(value) > 1:
            return value
        return value if isXML else value[0]
    if is_cheif_examiner:
        return " TBA " if isXML else "TBA"
    return " NA " if isXML else "NA"

#Function used to extract each unit blocks
def isNewBlock(line):
    res = re.findall(r'<div class="hbk-banner-box">', line)
    return not isEmpty(res)

#function used to extract unit ID of each unit block
def extractUnitID(line, isXML = False):
    res = re.findall(r'<span class="unitcode">\w*([a-zA-Z]{3}[0-9]{4})</span>', line)
    res = handleSingleValueCase(res,isXML = isXML)
    return res[0] if isXML else res

#Function used to extract unit Title of each unit block
def extractTitle(line, isXML = False):
    res = re.findall(r'</span> - (.*?)<span class="hbk-archive-only">', line, re.DOTALL)
    res = handleLinks(res) # all the links in this data is been removed
    res = handleSingleValueCase(res,isXML = isXML)
    return " " + res[0] if isXML else res

#Function used to extract unit Synopsis of each unit block
def extractSynopsis(line, isXML = False):
    res = re.findall(r'>Synopsis<(?:.*?)<p>(.*?)</p>', line, re.DOTALL)
    res = handleLinks(res) # all the links in this data is been removed
    res = handleSingleValueCase(res,isXML = isXML)
    return " " + res[0] if isXML else res

#Function used to extract unit PreRequisites of each unit block
def extractPrerequisites(line, isXML = False):
    #Both pre requisites and the co requisites is been extratced and combined together
    pre_res = re.findall(r'>Prerequisites<[\w\s\n/<>]+<p>(.*?)</p>', line,re.DOTALL)
    co_res = re.findall(r'>Co-requisites<[\w\s\n/<>]+<p>(.*?)</p>', line,re.DOTALL)
    res = []
    if not isEmpty(pre_res):
        pre_res = handleLinks(pre_res)
        pre_res = re.findall(r'([a-zA-Z]{3}[0-9]{4})',pre_res[0])
        res += pre_res
    if not isEmpty(co_res):
        co_res = handleLinks(co_res)
        co_res = re.findall(r'(?<!\()([a-zA-Z]{3}[0-9]{4})', co_res[0])
        res+= co_res
    if not isEmpty(res):
        res = list(set(res))
    return handleSingleValueCase(res,isXML = isXML)

#Function Used to extract Prohibitions
def extractProhibitions(line, isXML = False):
    #Extratcing all details in the <p> </p> after prohibitions
    res = re.findall(r'>Prohibitions</p>[\n]+<p>(?!None)(.*?)</p>', line)
    if not isEmpty(res):
        res = handleLinks(res) #Removing all the links
        res = re.findall(r'([a-zA-Z]{3}[0-9]{4})', res[0]) #Extracting only the unitIDs
        res = list(set(res))
    return handleSingleValueCase(res,isXML = isXML)

#Function used to extract Requirements
def extractRequirements(line, isXML = False):
    #Fincdling all values b/w <div> and </div>
    temp = re.findall(r'>Assessment<(?:.*?)<div>(.*?)</div>?', line,re.DOTALL)
    res = []
    if not isEmpty(temp):
        #From those results extracting values b/w <p> and </p>
        res = re.findall(r'<p>(.*?)</p>',temp[0],re.DOTALL)
        res = handleLinks(res) #all the unnecessary links is been removed
    return handleSingleValueCase(res,isXML = isXML)

#Function used to extract Outcomes
def extractOutcomes(line, isXML = False):
    #All the values b/w <div> and <div> ie the respective outcome block ie been extracted
    temp = re.findall(r'>Outcomes<(?:.|\n)*?<div>(.*?)</div>?', line,re.DOTALL)
    res = []
    if not isEmpty(temp):
        outcomesStr = temp[0]
        #Handling cases where <li> was present b/w a <ul>
        #in that case all the values whithin the ul is extarcted and appended along with the previous <li> point
        #And also those points are added again along to the rest of the list
        sub_set_temp = re.findall(r'(.*)<ul>(.*?)</ul>(.*)', outcomesStr,re.DOTALL)
        if not isEmpty(sub_set_temp):
            outcomesStr = ""
        for each in sub_set_temp:
            if not isEmpty(each):
                outcomesStr += each[0]
                temp = [each[1]]
                if not isXML:
                    temp = re.findall(r'(.*)</li>', each[1],re.DOTALL)
                    temp += "".join(handleLinks(temp))
                outcomesStr += handleLinks(temp)[0]
                outcomesStr += ("</li>"+each[1])
                outcomesStr += each[2]
        res = re.findall(r'<li>(.*?)</li>',outcomesStr,re.DOTALL)
        res = handleLinks(res)
    return handleSingleValueCase(res,isXML = isXML)

#Function used to extarct Cheif Examiner
def extractChiefExaminer(line, isXML = False):
    #vlaues in the cheif examiner block is been extracted
    res = re.findall(r'>Chief\sexaminer\(s\)<(?:.|\n)*?</p>', line)
    if not isEmpty(res):
        res = re.findall(r'>([\w\s\'-]+)</a>',res[0])
    res = handleLinks(res)
    return handleSingleValueCase(res,True,isXML = isXML)

#Function used to extarcting Each unit block and processing it
def extractUnitBlock(txtfile,isDecodingRequired = False,isXML = False):
    entireBlockStr = ""
    processedOnce = False #used to skip the first case where it detects the pattern
    res = []
    for line in txtfile:
        if not isNewBlock(line) and processedOnce:
            entireBlockStr += ("\n" + line)
        elif isNewBlock(line):
            processedOnce = True
            if len(entireBlockStr) != 0:
                res = handleBlockProcessing(entireBlockStr, res, isDecodingRequired, isXML)
                entireBlockStr = ""
    res = handleBlockProcessing(entireBlockStr, res, isDecodingRequired, isXML)
    return res

#Fucntion used to extract the list of unit blocks after processing
def handleBlockProcessing(entireBlockStr,result_array, isDecodingRequired=False, isXML = False):
    if len(entireBlockStr) != 0:
        temp = processBlock(entireBlockStr, isDecodingRequired,isXML)
        if not temp.isEmpty():
            result_array.append(temp)
    return result_array

#Function used to Process each unit Block
def processBlock(entireBlockStr,isDecodingRequired = False, isXML = False):
    entireBlockStr = re.sub(r'\n\n', "\n", entireBlockStr)
    if isDecodingRequired:
        #If decoding is required symbols like & etc is been decoded in the entire block string
        entireBlockStr = entireBlockStr.replace("&amp;", "&")
        entireBlockStr = entireBlockStr.replace("&gt;", ">")
        entireBlockStr = entireBlockStr.replace("&lt;", "<")
    else:
        #else handling case of xml
        entireBlockStr = entireBlockStr.replace("&amp;", "&#38;")
    
    #Respective block details is been processed and assigned
    block = UnitBlock()
    block.set_unit_id(extractUnitID(entireBlockStr,isXML))
    if block.isEmpty():
        return block
    block.set_title(extractTitle(entireBlockStr,isXML))
    block.set_synopsis(extractSynopsis(entireBlockStr,isXML))
    block.set_pre_requistics(extractPrerequisites(entireBlockStr,isXML))
    block.set_prohibisions(extractProhibitions(entireBlockStr,isXML))
    block.set_requirements(extractRequirements(entireBlockStr,isXML))
    block.set_outcomes(extractOutcomes(entireBlockStr,isXML))
    block.set_chief_examiners(extractChiefExaminer(entireBlockStr,isXML))
    #Entire details in the block is set to a dictionary
    block.configure_dict()
    
    return block

In [4]:
#Function Used for generating XML file after processing a Text file
def generateXML(file_to_read,writeFileName):
    read_file = open(file_to_read,"r")
    unitBlockObjects = extractUnitBlock(read_file,isXML = True)
    UnitBlock.writeToXML(unitBlockObjects,writeFileName)
    read_file.close()
    return

#Function Used for generating JSON file after processing a Text file
def generateJSON(readFileName, writeFileName):
    inFile = open(readFileName,"r")
    unitBlockObjects = extractUnitBlock(inFile,True)
    tempDictionary = UnitBlock.to_dict(unitBlockObjects)
    jsonStr = dumps(tempDictionary, indent=5)
    try:
        outFile = open(writeFileName, "w+")
        outFile.write(jsonStr)
        outFile.close()
    except:
        print("Exception occured while writing to json file")
    inFile.close()
    return

## Generating the Json File 

In [5]:
generateJSON("29566428.txt","29566428.json")

## Generating the XML File 

In [6]:
generateXML("29566428.txt","29566428.xml")